In [2]:
import pandas as pd
import json
from gpt3_utils import *

import numpy as np
import lime
import torch
import torch.nn.functional as F
from lime.lime_text import *
import scipy
import itertools
import ipdb

In [109]:
alternative_runs = []

In [317]:
pd.set_option('max_columns',40,'max_colwidth',400,'max_rows',100)

In [4]:
#Define which dev set output to load to run GPT-3 on
#Run test model first

run_dir = '../outputs/DDI/gpt3/0/cv/0/'
dev_ddi = pd.read_csv(run_dir+'gpt3.output.csv',sep='\t',index_col=0)

In [5]:
params = json.load(open(run_dir+'params.json','r'))

In [6]:
label_verbalizer = params['label_verbalizer']
label_list = np.sort(list(label_verbalizer.keys()),kind='stable')
verbalized_labels = [label_verbalizer[l] for l in label_list]
first_token_verbalized_labels = [tokenizer.decode(tokenizer.encode(" " + label)[0]) for label in verbalized_labels]

In [7]:
in_context_rel_dist = []
true_label_percentage = []

for i,row in dev_ddi.iterrows():
    final_input = row.final_input_prompts
    
    prompt_rels = []

    lines = final_input.split('\n')
    for line in lines:
        if 'Interaction:' in line:

            rel = line.split('Interaction:')[-1]
            if rel.strip() != '':
                prompt_rels.append(rel.strip())

    in_context_rel_dist.append(prompt_rels)
    true_label_percentage.append(len(np.where(row.verbalized_label == np.array(prompt_rels))[0])/len(prompt_rels))
    
dev_ddi['in_context_rel_dist'] = in_context_rel_dist
dev_ddi['true_label_percentage'] = true_label_percentage

In [8]:
knn_results = []
knn_correct_pseudo = []
knn_correct = []

for i,row in dev_ddi.iterrows():
    prompt_rels = row.in_context_rel_dist
    
    prompt_rels = pd.DataFrame(prompt_rels,columns=['rel'])
    prompt_rels['count'] = 1
    
    prompt_rels = prompt_rels.groupby('rel').count()
    max_rel = prompt_rels['count'].max()
    
    selected_rels = prompt_rels[prompt_rels['count'] == max_rel].index.values
    
    knn_results.append(selected_rels)
    
    pred = row.predictions
    
    if pred in selected_rels:
        if len(selected_rels) == 1:
            knn_correct.append(1)
        else:
            knn_correct.append(0)
        knn_correct_pseudo.append(1/len(selected_rels))
    else:
        knn_correct.append(0)
        knn_correct_pseudo.append(0)

In [9]:
dev_ddi['knn_results'] = knn_results
dev_ddi['knn_correct_pseudo'] = knn_correct_pseudo
dev_ddi['knn_correct'] = knn_correct

In [10]:
dev_ddi['gpt3_correct'] = dev_ddi.predictions == dev_ddi.verbalized_label

In [282]:
testing = dev_ddi.loc[[35]]

In [196]:
#If changing the in-context examples

prompt_to_choose = [5,1,2,3,4] #List of indices to use as prompt

new_test_ready_prompts = []

for i, row in testing.iterrows():
    
    samples = [dev_ddi.loc[ind].prompts for ind in prompt_to_choose]
                
    new_test_prompt = samples
    new_test_prompt.append(row.empty_prompts)
    new_test_prompt = '\n\n'.join(new_test_prompt)
    
    new_test_ready_prompts.append(new_test_prompt)

In [119]:
#If changing the text of the test sample

new_text = "Other strong inhibitors of CYP3A4 (e.g., itraconazole, clarithromycin, nefazodone, troleandomycin, ritonavir, nelfinavir) would be ____ ____ ____ similarly."

new_test_ready_prompts = []
new_empty_prompts = []

for i, row in testing.iterrows():
    sent = row['sents'] 
    
    full_prompt = row['test_ready_prompts']
    full_prompt = full_prompt.replace(sent, new_text)
    
    empty_prompt = row['empty_prompts']
    empty_prompt = empty_prompt.replace(sent, new_text)
    
    new_test_ready_prompts.append(full_prompt)
    new_empty_prompts.append(empty_prompt)

In [284]:
#If changing the text of the test sample and the prompt
new_full_prompt = """Sentence: Ascorbic acid: Doses of ascorbic acid (vitamin C) 1 g/day increase plasma concentration of synthetic estrogens by ~47%, possibly by inhibiting conjugation;
Drug 1: vitamin C
Drug 2: synthetic estrogens
Interaction: mechanism

Sentence: Interaction of gentamycin and atracurium in anaesthetised horses.  
Drug 1: gentamycin
Drug 2: atracurium
Interaction: other

Sentence: If desipramine hydrochloride is to be combined with other psychotropic agents such as tranquilizers or sedative/hypnotics, careful consideration should be given to the pharmacology of the agents employed since the sedative effects of desipramine and benzodiazepines (e.g., chlordiazepoxide or diazepam) are additive.
Drug 1: desipramine
Drug 2: benzodiazepines
Interaction: effect

Sentence: Isoflurane or enflurane administered with nitrous oxide/oxygen to achieve 1.25 MAC [Minimum Alveolar Concentration] may prolong the clinically effective duration of action of initial and maintenance doses of NIMBEX and decrease the required infusion rate of NIMBEX.
Drug 1: oxygen
Drug 2: NIMBEX
Interaction: none

Sentence: Levothyroxine Sodium Absorption: The following agents may bind and decrease absorption of clarithromycin from the gastrointestinal tract: aluminum hydoxide, cholestyramine resin, colestipol hydrochloride, ferrous sulfate, sodium polystyrene sulfonate, soybean flour (e.g., infant formula), sucralfate.
Drug 1: clarithromycin
Drug 2: sucralfate
Interaction:
"""
new_text = """Sentence: Levothyroxine Sodium Absorption: The following agents may bind and decrease absorption of clarithromycin from the gastrointestinal tract: aluminum hydoxide, cholestyramine resin, colestipol hydrochloride, ferrous sulfate, sodium polystyrene sulfonate, soybean flour (e.g., infant formula), sucralfate.
Drug 1: clarithromycin
Drug 2: sucralfate
Interaction:
"""

new_test_ready_prompts = []
new_empty_prompts = []

for i, row in testing.iterrows():
    sent = row['sents'] 
    
    full_prompt = new_full_prompt
    full_prompt = full_prompt.replace(sent, new_text)
    
    empty_prompt = new_text
    
    new_test_ready_prompts.append(full_prompt)
    new_empty_prompts.append(empty_prompt)

In [285]:
print(testing.test_ready_prompts.values[0])
print(testing.empty_prompts.values[0])
print('='*80)
print(new_test_ready_prompts[0])
print(new_empty_prompts[0])

testing['test_ready_prompts'] = new_test_ready_prompts
testing['empty_prompts'] = new_empty_prompts

Sentence: Ascorbic acid: Doses of ascorbic acid (vitamin C) 1 g/day have been reported to increase plasma concentration of synthetic estrogens by ~47%, possibly by inhibiting conjugation;
Drug 1: vitamin C
Drug 2: synthetic estrogens
Interaction: mechanism

Sentence: Interaction of gentamycin and atracurium in anaesthetised horses.  
Drug 1: gentamycin
Drug 2: atracurium
Interaction: other

Sentence: If desipramine hydrochloride is to be combined with other psychotropic agents such as tranquilizers or sedative/hypnotics, careful consideration should be given to the pharmacology of the agents employed since the sedative effects of desipramine and benzodiazepines (e.g., chlordiazepoxide or diazepam) are additive.
Drug 1: desipramine
Drug 2: benzodiazepines
Interaction: effect

Sentence: Agents that are CYP3A4 inhibitors that have been found, or are expected, to increase plasma levels of EQUETROTM are the following: Acetazolamide, azole antifungals, cimetidine, clarithromycin(1), dalfopri

In [286]:
new_results = run_gpt3_re_df(testing, params)

1it [00:00,  1.00it/s]
